required packages:
- ollama (external)
- langchain 
- langchain-community 
- langgraph
- yfinance
- faiss-cpu

In [18]:
a=    {'Equity Tech Fund': 15.0, 'Green Energy ETF': 10.0, 'Health Bio Stocks': 10.0, 'Global Bonds Fund': 15.0, 'CryptoIndex': 10.0, 'Real Estate REIT': 15.0, 'Emerging Markets Fund': 10.0, 'AI & Robotics ETF': 5.0, 'Commodities Basket': 5.0, 'Cash Reserve': 5.0}

In [19]:
sum(a.values())

100.0

In [27]:
a.values()  # returns a view object that displays a list of all the values in the dictionary
sum(a.values())  # returns the sum of all the values in the dictionary

100.0

In [37]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import os
from uuid import uuid4

# Funciones por separado

## YahooFinanceNewsTool

In [2]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.llms import Ollama
llm = Ollama(model='mistral')
tools = [YahooFinanceNewsTool()]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\piery\AppData\Local\Temp\ipykernel_46320\1286122019.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='mistral')
C:\Users\piery\AppData\Local\Temp\ipykernel_46320\1286122019.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/la

In [13]:
agent_chain.invoke(
    "What happened today with btc?",
)



> Entering new AgentExecutor chain...
 I need to find news about Bitcoin (BTC). To do this, I will use yahoo_finance_news.

Action: yahoo_finance_news
Action Input: BTC
Observation: Tap Into the Burgeoning Bitcoin Economy with Fold Holdings – Initiation Report
By Rayk Riechmann From banks to hedge funds to mom-and-pop investors, the Bitcoin (BTC) boom has garnered support from virtually every direction, with President Donald Trump this week vowing to create a strategic crypto reserve for the U.S. government. What if investors could bet on growth across the new Bitcoin economy while also getting direct […]

The Case for Accepting MicroStrategy’s (MSTR) Billion Dollar Bitcoin Gamble
The insatiable Bitcoin-buyer MicroStrategy (MSTR) has seen an impressive rise of over 600% in the past year—a performance roughly 4.5 times greater than Bitcoin’s (BTC) own gains over the same period. However, the rally has faced some headwinds since the end of last year, as MicroStrategy’s stock has strugg

{'input': 'What happened today with btc?',
 'output': "Today's developments with BTC include President Trump planning a strategic crypto reserve for the U.S., and the company MicroStrategy (MSTR) experiencing significant gains in the past year from its Bitcoin investments but facing challenges due to equity dilution concerns."}

## FAISS

In [ ]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Paso 1: Obtener noticias con la herramienta de LangChain
news_tool = YahooFinanceNewsTool()
noticias = news_tool.run("TSLA")  # También podrías usar 'AAPL', 'AMZN', etc.

# Paso 2: Separar en chunks si es necesario
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=40)
documentos = splitter.create_documents([noticias])

# Paso 3: Crear embeddings con Ollama (modelo local)
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# Paso 4: Guardar en base de datos FAISS (CPU)
vectorstore = FAISS.from_documents(documentos, embedding_model)

# Paso 5: Guardar en disco
vectorstore.save_local("faiss_news_tsla")

C:\Users\piery\AppData\Local\Temp\ipykernel_48904\1192078869.py:16: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="nomic-embed-text")


In [29]:
print(documentos)

[Document(metadata={}, page_content='Tesla, Inc. (TSLA): Seen as an ‘Embodied AI Secular Winner’'), Document(metadata={}, page_content='We recently published a list of 10 AI Stocks to Watch Amid Market Volatility. In this article, we are going to take a look at where Tesla, Inc. (NASDAQ:TSLA) stands against other AI stocks to watch amid market volatility. The uncertain macroeconomic environment is diminishing investors’ readiness'), Document(metadata={}, page_content='is diminishing investors’ readiness to pay high prices for stocks related to […]'), Document(metadata={}, page_content="Elon Musk, Jack Dorsey Call For Abolition Of All Intellectual Property Laws, Arguing There Are 'Much Greater Models To Pay Creators'"), Document(metadata={}, page_content='Elon Musk, CEO of Tesla Inc. (NASDAQ:TSLA), and Jack Dorsey, co-founder of Twitter (currently X) as well as Square, now called Block Inc. (NYSE:XYZ), have ignited a debate over intellectual property rights. The tech magnates have sugge

In [ ]:
# Cargar el índice desde disco
vectorstore = FAISS.load_local("./faiss_news_tsla/", embedding_model,allow_dangerous_deserialization=True)

# Hacer una búsqueda semántica
query = "¿Qué dice la prensa sobre Tesla y la energía solar?"
resultados = vectorstore.similarity_search(query, k=4)

# Mostrar los resultados
for i, r in enumerate(resultados, 1):
    print(f"🔍 Resultado {i}:\n{r.page_content}\n")

🔍 Resultado 1:
Tesla, Inc. (TSLA): Seen as an ‘Embodied AI Secular Winner’

🔍 Resultado 2:
We recently published a list of 10 AI Stocks to Watch Amid Market Volatility. In this article, we are going to take a look at where Tesla, Inc. (NASDAQ:TSLA) stands against other AI stocks to watch amid market volatility. The uncertain macroeconomic environment is diminishing investors’ readiness

🔍 Resultado 3:
Elon Musk, Jack Dorsey Call For Abolition Of All Intellectual Property Laws, Arguing There Are 'Much Greater Models To Pay Creators'

🔍 Resultado 4:
Elon Musk, CEO of Tesla Inc. (NASDAQ:TSLA), and Jack Dorsey, co-founder of Twitter (currently X) as well as Square, now called Block Inc. (NYSE:XYZ), have ignited a debate over intellectual property rights. The tech magnates have suggested a complete eradication of IP laws, a move that has stirred



## buscar noticias e indexar

experimental

In [ ]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.documents import Document

class CustomYahooFinanceNewsTool(YahooFinanceNewsTool):
    @staticmethod
    def _format_results(docs, query: str) -> str:
        # Personalizar el formato de los resultados para incluir más metadatos
        doc_strings = [{
            
                "Title": {doc.metadata['title']},
                "Description": {doc.metadata.get('description', '')},
                "URL": {doc.metadata.get('source', 'No URL available')}
        }
            
            for doc in docs
            if query in doc.metadata.get("description", "") or query in doc.metadata["title"]
        ]
        return doc_strings


real code

In [40]:
def actualizar_faiss_con_noticias(simbolo: str, 
                                   path_indice: str = "./faiss_news_index/", 
                                   modelo_ollama: str = "nomic-embed-text",
                                   chequear_duplicados: bool = True) -> None:
    import hashlib

    def hash_text(text: str) -> str:
        return hashlib.md5(text.encode('utf-8')).hexdigest()

    print(f"Obteniendo noticias para: {simbolo}")
    embedding_model = OllamaEmbeddings(model=modelo_ollama)
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=40)
    news_tool = YahooFinanceNewsTool()
    
    # 1. Obtener noticias
    try:
        noticias = news_tool.run(simbolo)
    except:
        print("Error al obtener noticias. Verifica el símbolo o la conexión a internet.")
        return

    if not noticias or "no news found" in noticias.lower():
        print("No se encontraron noticias relevantes.")
        return

    # 2. Separar en documentos y filtrar vacíos
    documentos_nuevos = splitter.create_documents([noticias])
    documentos_nuevos = [doc for doc in documentos_nuevos if doc.page_content.strip() != ""]
    
    if not documentos_nuevos:
        print("No hay contenido útil para indexar.")
        return

    contenido_existente = set()

    # 3. Cargar índice existente si existe
    if os.path.exists(path_indice):
        print("Cargando índice existente...")
        vectorstore = FAISS.load_local(path_indice, embedding_model, allow_dangerous_deserialization=True)
        if chequear_duplicados:
            documentos_existentes = vectorstore.docstore._dict.values()
            contenido_existente = set(hash_text(doc.page_content) for doc in documentos_existentes)
    else:
        print("Creando nuevo índice FAISS...")
        vectorstore = FAISS.from_documents(documentos_nuevos, embedding_model)

    # 4. Filtrar duplicados (si corresponde)
    if chequear_duplicados:
        documentos_para_agregar = [doc for doc in documentos_nuevos if hash_text(doc.page_content) not in contenido_existente]
    else:
        documentos_para_agregar = documentos_nuevos

    if not documentos_para_agregar:
        print("Las noticias ya estaban indexadas. No se agregaron documentos nuevos.")
        return

    # 5. Generar embeddings y agregar
    new_embeddings = embedding_model.embed_documents([doc.page_content for doc in documentos_para_agregar])
    ids = [str(uuid4()) for _ in documentos_para_agregar]

    vectorstore.add_documents(documents=documentos_para_agregar, ids=ids)

    # 6. Guardar cambios
    vectorstore.save_local(path_indice)
    print(f"Índice actualizado con {len(documentos_para_agregar)} nuevas noticias de {simbolo}.")


In [124]:
actualizar_faiss_con_noticias("TSLA", path_indice="./faiss_news_index/")

Obteniendo noticias para: TSLA
Creando nuevo índice FAISS...
Índice creado.


In [ ]:
vectorstore = FAISS.load_local("./faiss_news_index/", embedding_model, allow_dangerous_deserialization=True)

query = "¿Qué dice la prensa sobre Tesla y la energía solar?"
resultados = vectorstore.similarity_search(query, k=20)
# Mostrar los resultados
for i, r in enumerate(resultados, 1):
    print(f"🔍 Resultado {i}:\n{r.page_content})\n")


🔍 Resultado 1:
URL: https://finance.yahoo.com/news/tesla-inc-tsla-seen-embodied-185057311.html)

🔍 Resultado 2:
Title: Tesla, Inc. (TSLA): Seen as an ‘Embodied AI Secular Winner’)

🔍 Resultado 3:
Description: We recently published a list of 10 AI Stocks to Watch Amid Market Volatility. In this article, we are going to take a look at where Tesla, Inc. (NASDAQ:TSLA) stands against other AI stocks to watch amid market volatility. The uncertain macroeconomic environment is diminishing)

🔍 Resultado 4:
environment is diminishing investors’ readiness to pay high prices for stocks related to […])



## responder_preguntas 

In [43]:
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

def crear_chain_de_preguntas_mistral(path_indice="./faiss_news_index/") -> RetrievalQA:
    """
    Crea una cadena de preguntas-respuestas usando un índice FAISS y Mistral 7B local.
    
    Args:
        path_indice (str): Ruta al índice FAISS.
    
    Returns:
        RetrievalQA: Cadena lista para responder preguntas.
    """
    print("Cargando modelo Mistral y vectorstore...")
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    llm = Ollama(model="mistral")  # o "mistral:instruct" si tienes ese tag cargado

    vectorstore = FAISS.load_local(path_indice, embeddings, allow_dangerous_deserialization=True)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    return chain


In [45]:
qa = crear_chain_de_preguntas_mistral()

while True:
    pregunta = input("¿En qué puedo ayudarte sobre el mercado?: ")
    if pregunta.lower() in ["salir", "exit", "q"]:
        break
    respuesta = qa(pregunta)
    print("\n📢 Respuesta:", respuesta["result"])
    print("\n📄 Fuentes:")
    for doc in respuesta["source_documents"]:
        print("•", doc.metadata.get("simbolo", "Desconocido"))


Cargando modelo Mistral y vectorstore...

📢 Respuesta:  Based on the current market trends and news, I would recommend three stocks with a positive outlook:

1. NVIDIA Corporation (NVDA) - According to recent reports, this tech giant is highly favored by billionaire investors. However, it's essential to note that investing always involves risk, and past performance is not indicative of future results.

2. Microsoft Corporation (MSFT) - Although not explicitly mentioned in the context provided, Microsoft has shown strong growth potential in areas such as cloud services, gaming, and AI technologies.

3. Amazon.com, Inc. (AMZN) - Despite facing increased competition and regulatory scrutiny, Amazon continues to innovate and expand its business model, making it a potentially promising investment for the long term.

As always, investors should conduct their own research or consult with a financial advisor before making any investment decisions.

📄 Fuentes:
• Desconocido
• Desconocido
• Desco

## analizar_portafolio

In [125]:
portafolio = {
    "AAPL": {
        "porcentaje": 0.4,
        "cantidad_acciones": 10,
        "precio_promedio": 175.30
    },
    "TSLA": {
        "porcentaje": 0.3,
        "cantidad_acciones": 5,
        "precio_promedio": 220.10
    },
    "MSFT": {
        "porcentaje": 0.3,
        "cantidad_acciones": 8,
        "precio_promedio": 315.50
    }
}


analizar tono

In [10]:
def clasificar_tono(texto: str, qa_chain) -> str:
    """
    Clasifica el sentimiento del texto como positivo, negativo o neutral.
    
    Args:
        texto (str): El texto a clasificar.
        qa_chain: Cadena RAG o LLM para inferencia.
    
    Returns:
        str: Uno de 'positivo', 'negativo' o 'neutral'.
    """
    prompt = f"""Clasifica el siguiente análisis de noticias como 'positivo', 'negativo' o 'neutral, basandote en su impacto en el mercado de valores.':
    
    Texto: \"\"\"{texto}\"\"\"
    
    Solo responde con una palabra: positivo, negativo o neutral."""
    
    try:
        respuesta = qa_chain({"query": prompt})
        return respuesta["result"].strip().lower()
    except Exception as e:
        return f"error: {str(e)}"


In [11]:
def analizar_portafolio(portafolio: dict, qa_chain) -> dict:
    analisis = {}

    for simbolo in portafolio.keys():
        print(f"🔍 Analizando {simbolo}...")
        query = f"¿Qué riesgos u oportunidades recientes hay respecto a la empresa {simbolo} según las últimas noticias financieras?"

        try:
            respuesta = qa_chain({"query": query})
            resumen = respuesta["result"]
            tono = clasificar_tono(resumen, qa_chain)
            
            analisis[simbolo] = {
                "resumen": resumen,
                "tono": tono
            }

        except Exception as e:
            analisis[simbolo] = {
                "resumen": f"❌ Error al analizar: {str(e)}",
                "tono": "error"
            }

    return analisis


In [42]:
portafolio = {
    "AMD": 0.4,
    "NVDA":  0.3,
    "F":  0.3
}

for i in portafolio.keys():
    actualizar_faiss_con_noticias(i, path_indice="./faiss_news_index/")
    # Actualiza el índice FAISS con noticias para cada símbolo en el portafolio

# qa_chain = crear_chain_de_preguntas_mistral()  # usando la función que definimos antes

# resultado = analizar_portafolio(portafolio, qa_chain)

# for simbolo, resumen in resultado.items():
#     print(f"\n📈 {simbolo}:")
#     print(resumen)


Obteniendo noticias para: AMD
Cargando índice existente...
Las noticias ya estaban indexadas. No se agregaron documentos nuevos.
Obteniendo noticias para: NVDA
Cargando índice existente...
Las noticias ya estaban indexadas. No se agregaron documentos nuevos.
Obteniendo noticias para: F
Cargando índice existente...
Las noticias ya estaban indexadas. No se agregaron documentos nuevos.


## recomendar ajustes

In [24]:
def recomendar_ajustes(analisis: dict, portafolio: dict) -> dict:
    """
    Propone ajustes al portafolio según el tono del análisis.

    Args:
        analisis (dict): Diccionario con 'resumen' y 'tono' por símbolo.
        portafolio (dict): Portafolio actual, ej: {"AAPL": 0.4, "TSLA": 0.6}

    Returns:
        dict: Ajustes propuestos, por ejemplo {"AAPL": "reducir", "TSLA": "mantener"}
    """
    recomendaciones = {}

    for simbolo, data in analisis.items():
        tono = data.get("tono", "neutral")

        if tono == "positivo" or tono == "positive":
            recomendaciones[simbolo] = "aumentar"
        elif tono == "negativo" or tono == 'negative':
            recomendaciones[simbolo] = "reducir"
        else:
            recomendaciones[simbolo] = "mantener"

    return recomendaciones


In [25]:
recomendaciones = recomendar_ajustes(resultado, portafolio)

## ejecutar ajuste

In [28]:
def ejecutar_ajuste(portafolio: dict, recomendaciones: dict,  step: float = 0.1) -> dict:
    """
    Aplica los ajustes al portafolio si el usuario confirma.

    Args:
        portafolio (dict): Estado actual del portafolio.
        recomendaciones (dict): {"AAPL": "reducir", "TSLA": "aumentar", ...}
        confirmacion_usuario (bool): Si el usuario acepta los cambios.
        step (float): Cuánto ajustar por símbolo (por defecto 10%).

    Returns:
        dict: Nuevo estado del portafolio.
    """
    confirmacion_usuario = input(f"¿Aplicar ajustes al portafolio? {recomendaciones} (s/n): ").strip().lower() == 's'
    if not confirmacion_usuario:
        print("Usuario rechazó los ajustes. Portafolio sin cambios.")
        return portafolio

    nuevo_portafolio = portafolio.copy()

    for simbolo, accion in recomendaciones.items():
        actual = nuevo_portafolio.get(simbolo, 0.0)

        if accion == "aumentar":
            nuevo_portafolio[simbolo] = min(actual + step, 1.0)
        elif accion == "reducir":
            nuevo_portafolio[simbolo] = max(actual - step, 0.0)
        # mantener → sin cambio

    # Normalizar
    total = sum(nuevo_portafolio.values())
    for k in nuevo_portafolio:
        nuevo_portafolio[k] /= total

    print("Ajustes aplicados.")
    return nuevo_portafolio


# Grafo LangGraph

In [46]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable

# Estado del grafo
class EstadoCartera(TypedDict):
    cartera: List[str]
    noticias_indexadas: bool

# Nodo: Bienvenida y construcción inicial de cartera
def bienvenida_y_cartera(state: EstadoCartera) -> tuple[EstadoCartera, AIMessage]:
    # Aquí se puede usar Tavily para buscar acciones populares
    print("✅ Bienvenida y generación de cartera inicial...")
    
    acciones_iniciales = ["AAPL", "TSLA", "NVDA", "MSFT", "AMZN"]  # mock o Tavily aquí
    mensaje = AIMessage(content=f"""
¡Hola! 👋 Bienvenido/a al asistente de inversiones.

Aquí tienes una cartera inicial sugerida basada en las acciones más populares del momento:
{acciones_iniciales}

¿Qué te gustaría hacer ahora?
- Ver noticias recientes 📰
- Consultar por un sector específico 🔍
- Ver estado de la cartera 📈
- Obtener sugerencias de rebalanceo 🔁
""")
    return {"cartera": acciones_iniciales, "noticias_indexadas": False}, mensaje


In [52]:
# Crear el grafo
graph = StateGraph(EstadoCartera)
graph.add_node("bienvenida_y_cartera", bienvenida_y_cartera)

# Transición directa al final por ahora
graph.set_entry_point("bienvenida_y_cartera")
graph.set_finish_point("bienvenida_y_cartera")  # solo para esta fase de prueba

# Compilar grafo
grafo_compilado = graph.compile()


In [53]:
estado_inicial = {"cartera": [], "noticias_indexadas": False}
resultado = grafo_compilado.invoke(estado_inicial)
print(resultado[1].content)


✅ Bienvenida y generación de cartera inicial...


InvalidUpdateError: Expected dict, got ({'cartera': ['AAPL', 'TSLA', 'NVDA', 'MSFT', 'AMZN'], 'noticias_indexadas': False}, AIMessage(content="\n¡Hola! 👋 Bienvenido/a al asistente de inversiones.\n\nAquí tienes una cartera inicial sugerida basada en las acciones más populares del momento:\n['AAPL', 'TSLA', 'NVDA', 'MSFT', 'AMZN']\n\n¿Qué te gustaría hacer ahora?\n- Ver noticias recientes 📰\n- Consultar por un sector específico 🔍\n- Ver estado de la cartera 📈\n- Obtener sugerencias de rebalanceo 🔁\n", additional_kwargs={}, response_metadata={}))
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [58]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
import re
import os
os.environ["TAVILY_API_KEY"] = "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"



def extraer_tickers_desde_resultados(resultados: list[str]) -> list[str]:
    """Extrae posibles tickers bursátiles desde resultados de búsqueda Tavily."""
    tickers = set()
    for texto in resultados:
        # Buscar tickers comunes: AAPL, TSLA, META, etc.
        posibles = re.findall(r'\b[A-Z]{2,5}\b', texto)
        tickers.update(posibles)
    return list(tickers)

def obtener_tickers_relevantes_tavily(query: str = "top performing stocks April 2025") -> list[str]:
    """Usa Tavily para buscar acciones destacadas y extraer tickers bursátiles."""
    tavily_tool = TavilySearchResults(k=5)
    resultados = tavily_tool.run(query)

    # `resultados` es una lista de dicts con 'content', 'url', etc.
    textos = [r['content'] for r in resultados if 'content' in r]
    tickers = extraer_tickers_desde_resultados(textos)
    return tickers


In [68]:
tavily_tool = TavilySearchResults(k=5)
resultados = tavily_tool.run("best stocks to buy for 2025 based on news")

# `resultados` es una lista de dicts con 'content', 'url', etc.
textos = [r['content'] for r in resultados if 'content' in r]

In [76]:
len(textos)

5

In [77]:


tickers = obtener_tickers_relevantes_tavily("best stocks to buy for 2025 based yahoo finances and news")
print("Tickers relevantes encontrados:", tickers)

Tickers relevantes encontrados: ['IONQ', 'AMZN', 'AI', 'NYSE', 'CZR', 'CEO', 'PDD', 'GPN', 'ACGL', 'BRK', 'MGM', 'COST', 'CMCSA', 'CAGR', 'SOFI', 'NU', 'WBD', 'EA', 'ALL', 'LVS']


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import Tool

os.environ["TAVILY_API_KEY"] = ""
# Configura Tavily
tavily_tool = TavilySearchResults()

# Lo puedes envolver como tool para usar dentro del LangGraph
tavily_langgraph_tool = Tool(
    name="Tavily Search",
    func=tavily_tool.run,
    description="Busca noticias relevantes sobre sectores financieros."
)


In [79]:
distribucion = {
    "Equity Tech Fund": 0.20,
    "GreenEnergy ETF": 0.15,
    "Health Bio Stocks": 0.10,
    "Global Bonds Fund": 0.10,
    "CryptoIndex": 0.10,
    "RealEstate REIT": 0.10,
    "Emerging Markets Fund": 0.10,
    "AI & Robotics ETF": 0.05,
    "Commodities Basket": 0.05,
    "Cash Reserve": 0.05
    
}

# Calcular montos
cartera = {k: round(v * 10000, 2) for k, v in distribucion.items()}


In [81]:
sectores = [
    "technology stocks",
    "renewable energy",
    "biotech stocks",
    "global bonds",
    "cryptocurrencies",
    "real estate REIT",
    "emerging markets",
    "AI and robotics",
    "commodities",
]

noticias_por_sector = {}
for sector in sectores:
    resultado = tavily_tool.run(sector)
    noticias_por_sector[sector] = resultado


In [86]:
noticias

"Tesla (NasdaqGS:TSLA) Reports First Quarter 2025 Vehicle Production & Delivery\nTesla (NasdaqGS:TSLA) reported its first quarter 2025 results on April 2, revealing the production of 362,615 vehicles and delivery of 336,681 vehicles, which aligns with its long-term operational goals. These robust figures may have bolstered investor confidence, contributing to a 15% gain in the company's share price over the past week. Despite the broader market downturn prompted by U.S. restrictions on chip exports to China affecting major tech stocks, Tesla managed to outperform the..."

In [ ]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

ollama_llm = Ollama(model="mistral")

template = PromptTemplate.from_template("""
Dada la siguiente distribución de cartera y las noticias recientes de cada sector, explica por qué esta es una buena estrategia para una inversión inicial de $10.000.

Cartera:
{cartera}

Noticias:
{noticias}

Justifica brevemente por qué cada categoría tiene la asignación que tiene.
""")

cadena_justificacion = LLMChain(llm=ollama_llm, prompt=template)

output = cadena_justificacion.run(
    cartera=cartera,
    noticias=noticias_por_sector
)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import Ollama
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from tavily import TavilyAPI  # Asegúrate de instalar la librería Tavily o la que uses
import os
os.environ["TAVILY_API_KEY"] = ""

# API Key para Tavily (o la API que utilices)

# Configurar LLM de Ollama (Mistral 7B)
llm = Ollama(model="mistral")

# Crear la herramienta para obtener noticias generales
def obtener_noticias_generales():
    tavily_client = TavilyAPI()

    # Hacer una consulta general (ajusta según la API y los parámetros)
    noticias = tavily_client.get_news(query="mercados financieros")
    return noticias['articles']

# Crear herramienta personalizada para obtener noticias generales
news_tool = Tool(
    name="Noticias Generales",
    func=obtener_noticias_generales,
    description="Obtiene noticias sobre mercados financieros, fondos y sectores de inversión"
)

# Crear la plantilla de bienvenida
prompt_bienvenida = PromptTemplate.from_template("""
    Hola, bienvenido a tu asistente financiero. Vamos a empezar con la creación de una cartera de inversión inicial.

    Tu inversión inicial es de $10,000 USD. Te voy a proporcionar una distribución recomendada en los siguientes instrumentos financieros:

    | Instrumento          | Descripción detallada                                                                                             | Alocación Inicial (%) |
    |----------------------|-------------------------------------------------------------------------------------------------------------------|-----------------------|
    | Equity Tech Fund     | Fondo de inversión compuesto por acciones de grandes empresas tecnológicas (software, hardware, nube), con alta volatilidad y crecimiento potencial. | 20%                   |
    | GreenEnergy ETF      | Fondo cotizado en bolsa (ETF) que invierte en compañías del sector de energías renovables como solar, eólica e hidroeléctrica, con crecimiento sostenible. | 15%                   |
    | Health Bio Stocks    | Acciones de empresas farmacéuticas y biotecnológicas con alta inversión en investigación y desarrollo, sujetas a regulaciones estrictas. | 10%                   |
    | Global Bonds Fund    | Fondo de bonos de deuda gubernamental de distintos países, proporcionando estabilidad y bajo riesgo en comparación con otros activos. | 10%                   |
    | CryptoIndex          | Portafolio compuesto por múltiples criptomonedas de alto capital de mercado, con alta volatilidad e influencia de noticias macroeconómicas. | 10%                   |
    | RealEstate REIT      | Fideicomiso de inversión inmobiliaria (REIT) que proporciona exposición a propiedades comerciales y residenciales, con flujos de ingresos estables. | 10%                   |
    | Emerging Markets Fund| Fondo de inversión que incluye activos de economías en desarrollo, con alto potencial de crecimiento pero también mayor riesgo geopolítico. | 10%                   |
    | AI & Robotics ETF    | ETF compuesto por empresas innovadoras en inteligencia artificial, automatización y robótica, con alto potencial disruptivo. | 5%                    |
    | Commodities Basket   | Fondo diversificado en materias primas, incluyendo metales preciosos y energía, con cobertura contra inflación y volatilidad en mercados tradicionales. | 5%                    |
    | Cash Reserve         | Fondos en efectivo o equivalentes de alta liquidez, usados para minimizar riesgo y proporcionar flexibilidad en inversiones. | 5%                    |

    A continuación, te mostraré cómo puedes acceder a noticias relevantes sobre estos fondos y otras oportunidades de inversión. ¿Te gustaría que te muestre algunas noticias relacionadas?
""")

# Crear el agente con el LLM y las herramientas
tools = [news_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Crear el flujo de trabajo para ejecutar el agente
def ejecutar_agente_con_grafo():
    result = agent.run(prompt_bienvenida)
    print(result)

# Ejecutar el flujo
ejecutar_agente_con_grafo()


## newsAPI

In [ ]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

# Reemplaza con tu API key
api_key = ""
base_url = "https://newsapi.org/v2/"
endpoint = "everything"
query = "Equity Tech Fund"  # Puedes cambiar esto por tus palabras clave
page_size=20
# Calcular la fecha de hace una semana (considerando la hora actual en Chile)
now_cl = datetime.now()
one_week_ago_cl = now_cl - timedelta(days=7)
from_date = one_week_ago_cl.strftime("%Y-%m-%d")

url = f"{base_url}{endpoint}?q={query}&from={from_date}&pageSize={page_size}&apiKey={api_key}"

response = requests.get(url)
data = response.json()

relevant_news = {}

if response.status_code == 200:
    articles = data.get("articles")
    if articles:
        print(f"Noticias de '{query}' de la última semana (hora de Chile):")
        for article in articles:
            article_url = article.get("url")
            print(f"Título: {article.get('title')}")
            print(f"URL: {article_url}")

            # Intentar obtener el contenido del artículo con varios tags
            content = ""
            try:
                article_response = requests.get(article_url)
                article_response.raise_for_status()
                soup = BeautifulSoup(article_response.text, 'html.parser')

                possible_containers = [
                    'article',
                    'div.article-body',
                    'div.main',
                    'div.content',
                    'div.story',
                    'div[role="main"]',  # Buscar divs con el rol "main"
                    'section.article-content',
                    'section.main-content'
                ]

                for selector in possible_containers:
                    container = soup.select_one(selector)
                    if container:
                        content = '\n'.join([p.get_text() for p in container.find_all('p')])
                        if content.strip():  # Si encontramos contenido, detenemos la búsqueda
                            break

                if not content.strip():
                    # Si no se encontró un contenedor específico, extraer texto de todos los párrafos
                    content = '\n'.join([p.get_text() for p in soup.find_all('p')])

                print(f"Contenido:\n{content[:500]}..." if len(content) > 500 else f"Contenido:\n{content}") # Mostrar solo los primeros 500 caracteres para no saturar la salida

            except requests.exceptions.RequestException as e:
                print(f"No se pudo obtener el artículo: {e}")
                content = None  # No se pudo obtener el contenido
            except AttributeError:
                print("No se pudo extraer el contenido del artículo (estructura HTML inesperada).")
                content = None

            if content:
                # Guardar el artículo en el diccionario solo si se obtuvo contenido
                relevant_news[article_url] = {
                    "title": article.get("title"),
                    "description": article.get("description"),
                    "content": content,
                    "source": article.get("source", {}).get("name", "Desconocido")
                }
            if len(relevant_news) >= 10:
                break
            print("-" * 20)

    else:
        print("No se encontraron noticias para los criterios especificados en la última semana.")
        content = None  # No se encontraron artículos
else:
    print(f"Error al hacer la solicitud: {response.status_code}")
    print(f"Mensaje de error: {data.get('message')}")

Noticias de 'Equity Tech Fund' de la última semana (hora de Chile):
Título: Meta and the FTC face off in court over monopoly claims
URL: https://www.npr.org/2025/04/14/nx-s1-5358436/meta-ftc-antitrust-trial-opening-arguments
Contenido:


      Bobby Allyn
    


                A photograph taken during the World Economic Forum (WEF) annual meeting in Davos in January shows the logo of Meta, the U.S. company that owns and operates Facebook, Instagram, Threads and WhatsApp.
                
                    
                    Fabrice Coffrini/AFP via Getty Images
                    
                
hide caption

Meta CEO Mark Zuckerberg defended his social media platform on Monday before a federal judge in the U.S. gover...
--------------------
Título: Poolhouse raises $34M for gamified pool from the creators of Topgolf
URL: https://venturebeat.com/games/poolhouse-raises-34m-for-gamified-pool-from-the-creators-of-topgolf/
No se pudo obtener el artículo: 403 Client Error: Forbidde